<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Наша задача - обучить модель и классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Модель должна быть со значением метрики качества *F1* не меньше 0.75. 

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

In [1]:
import pandas as pd
import numpy as np
import re
from pymystem3 import Mystem
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
import nltk
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
import spacy
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import RandomizedSearchCV
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import f1_score

In [2]:
df = pd.read_csv('/datasets/toxic_comments.csv', index_col = 0)

In [3]:
df.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


Видим, что в данных присутствует разделитель. Избавимся от него.

In [5]:
def clean(text):
    text = re.sub(r"(?:\n|\r)", " ", text)
    text = re.sub(r"[^a-zA-Z ]+", "", text).strip()
    text = text.lower()
    return text

df['text'] = df['text'].apply(clean)

Проведем лемматизацию.

In [6]:
nltk.download('wordnet')
lemmatizer = WordNetLemmatizer()

def lemmatize(text):
    words = text.split()
    lemmatized_words = [lemmatizer.lemmatize(word, wordnet.VERB) for word in words]
    lemmatized_text = ' '.join(lemmatized_words)
    return lemmatized_text

df['text'] = df['text'].apply(lemmatize)
df['text']

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


0         explanation why the edit make under my usernam...
1         daww he match this background colour im seemin...
2         hey man im really not try to edit war its just...
3         more i cant make any real suggestions on impro...
4         you sir be my hero any chance you remember wha...
                                ...                        
159446    and for the second time of ask when your view ...
159447    you should be ashamed of yourself that be a ho...
159448    spitzer umm theres no actual article for prost...
159449    and it look like it be actually you who put on...
159450    and i really dont think you understand i come ...
Name: text, Length: 159292, dtype: object

## Обучение

Обозначим стоп-слова.

In [7]:
stop_words = set(stopwords.words('english'))

In [8]:
y = df['toxic'].values
X = df['text'].values
TEST_SIZE = 0.1
VALIDATION_SIZE = 0.1
RANDOM_STATE = 42

Теперь мы готовы к подбору лучшей модели Логистической регресии.

In [9]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=TEST_SIZE + VALIDATION_SIZE, random_state=RANDOM_STATE)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=TEST_SIZE / (TEST_SIZE + VALIDATION_SIZE), random_state=RANDOM_STATE)

tfidf = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 1))
X_train_tfidf = tfidf.fit_transform(X_train)
X_val_tfidf = tfidf.transform(X_val)
X_test_tfidf = tfidf.transform(X_test)

warnings.filterwarnings("ignore", category=UserWarning, module='sklearn')

In [10]:
best_f1 = 0
best_params = {}

for C in [1, 5, 10]:
    for penalty in ['l1', 'l2']:
        clf = LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced', solver='saga', max_iter=100, C=C, penalty=penalty)
        clf.fit(X_train_tfidf, y_train)
        y_val_pred = clf.predict(X_val_tfidf)
        f1 = f1_score(y_val, y_val_pred)
        print(f'C={C}, penalty={penalty}, f1={f1}')
        if f1 > best_f1:
            best_f1 = f1
            best_params = {'C': C, 'penalty': penalty}

C=1, penalty=l1, f1=0.6971141781681305
C=1, penalty=l2, f1=0.7390946502057613
C=5, penalty=l1, f1=0.7462600690448792
C=5, penalty=l2, f1=0.7684992570579494
C=10, penalty=l1, f1=0.7586626139817629
C=10, penalty=l2, f1=0.7579856115107914


In [11]:
best_clf = LogisticRegression(random_state=RANDOM_STATE, class_weight='balanced', solver='saga', max_iter=100, **best_params)
best_clf.fit(X_train_tfidf, y_train)

y_val_pred = best_clf.predict(X_val_tfidf)
final_f1 = f1_score(y_val, y_val_pred)

print(f'Лучшие параметры для модели: {best_params}')
print(f'Финальная метрика F1 на валидационной выборке: {final_f1}')

Лучшие параметры для модели: {'C': 5, 'penalty': 'l2'}
Финальная метрика F1 на валидационной выборке: 0.7684992570579494


Сделаем тоже самое для случайного леса.

In [12]:
best_f1 = 0
best_params = {}

for n_estimators in [50, 300, 50]:
    for max_depth in [5, 15]:
        clf_2 = RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced', n_estimators=n_estimators, max_depth=max_depth)
        clf_2.fit(X_train_tfidf, y_train)
        y_val_pred = clf_2.predict(X_val_tfidf)
        f1 = f1_score(y_val, y_val_pred)
        print(f'n_estimators={n_estimators}, max_depth={max_depth}, f1={f1}')
        if f1 > best_f1:
            best_f1 = f1
            best_params = {'n_estimators': n_estimators, 'max_depth': max_depth}

n_estimators=50, max_depth=5, f1=0.32015200095000595
n_estimators=50, max_depth=15, f1=0.3555908850026498
n_estimators=300, max_depth=5, f1=0.3288416075650118
n_estimators=300, max_depth=15, f1=0.37113402061855666
n_estimators=50, max_depth=5, f1=0.32015200095000595
n_estimators=50, max_depth=15, f1=0.3555908850026498


In [13]:
best_clf_2 = RandomForestClassifier(random_state=RANDOM_STATE, class_weight='balanced', **best_params)
best_clf_2.fit(X_train_tfidf, y_train)

y_val_pred = best_clf_2.predict(X_val_tfidf)
final_f1 = f1_score(y_val, y_val_pred)

print(f'Лучшие параметры для модели: {best_params}')
print(f'Финальная метрика F1 на валидационной выборке: {final_f1}')

Лучшие параметры для модели: {'n_estimators': 300, 'max_depth': 15}
Финальная метрика F1 на валидационной выборке: 0.37113402061855666


Согласно оценкам, видим, что регрессия оказалась лучше. Выплоним тест на тестовых данных, и сделаем выводы.

In [14]:
y_test_pred = best_clf.predict(X_test_tfidf)
final_f1 = f1_score(y_test, y_test_pred)

print(f'Финальная метрика F1 на тестовой выборке: {final_f1}')

Финальная метрика F1 на тестовой выборке: 0.7771156138259833


## Выводы

В ходе выполнения задачи мы провели предобработку данных, включающую удаление разделителей и лемматизацию текста. Затем мы обучили модели, и оценили ее производительность с помощью метрики F1.<br><br>Итоговый результат показывает, что логистическая регрессия показала себя лучше, ведь ее значение F1 на тестовом наборе данных равно 0.777, что удовлетворяет требованию поставленной задачи. Это означает, что модель способна эффективно классифицировать комментарии на позитивные и негативные, что позволит магазину автоматически отправлять токсичные комментарии на модерацию.<br><br>Таким образом, мы успешно разработали инструмент для обнаружения токсичных комментариев, что поможет улучшить качество контента и обеспечить более безопасное и приятное пользовательское взаимодействие в интернет-магазине "Викишоп".